# Construction and validation of bioNLP named entity recognition model using deep learning
Problems: the embeddings I trained is worse than the one from google

deep_nlp_serial

Attribution: 
    Some of the code are borrowed from the example code from spacy

In [1]:
import tensorflow
from tqdm import tqdm

import keras

import random
import pathlib
import cytoolz
import numpy
from keras.models import Sequential, model_from_json
from keras.layers import LSTM, Dense, Embedding, Bidirectional
from keras.layers import TimeDistributed
from keras.optimizers import Adam
import thinc.extra.datasets
from spacy.compat import pickle
import spacy
import pandas as pd
from sklearn import preprocessing
import keras
from sklearn import model_selection
#import seaborn as sns
from sklearn import metrics
#import matplotlib.pyplot as plt

Using TensorFlow backend.
/cellar/users/btsui/anaconda3/envs/deep_nlp_cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/cellar/users/btsui/anaconda3/envs/deep_nlp_cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


### load in SRA data

Wall time: 47.5 s


In [2]:
%%time 
inS_dir='/cellar/users/btsui/Data/nrnb01_nobackup/METAMAP/allSRS.pickle'
sra_dump_pickle_dir='/cellar/users/btsui/Data/SRA/DUMP/sra_dump.pickle'
srsS=pd.read_pickle(inS_dir)
technical_meta_data_df=pd.read_pickle(sra_dump_pickle_dir)


CPU times: user 9.93 s, sys: 3.54 s, total: 13.5 s
Wall time: 13.5 s


### load in spacy models

Wall time: 52.1 s


In [3]:
%%time
#nlp=spacy.load('en_vectors_web_lg')
nlp=spacy.load('./wikipedia-pubmed-and-PMC-w2v')

/cellar/users/btsui/anaconda3/envs/deep_nlp_cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/cellar/users/btsui/anaconda3/envs/deep_nlp_cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


CPU times: user 46.8 s, sys: 5.68 s, total: 52.4 s
Wall time: 52.4 s


### Functions for model compilation and feature extraction

Useful information for understanding the neural network: 

TimeDistributed: https://machinelearningmastery.com/timedistributed-layer-for-long-short-term-memory-networks-in-python/

lr stands for: learning rate

In [4]:
def compile_lstm(embeddings, shape, settings):
    model = Sequential()
    model.add(
        Embedding(
            embeddings.shape[0],
            embeddings.shape[1],
            input_length=shape['max_length'],
            trainable=False,
            weights=[embeddings],
            mask_zero=True
        )
    )
    #the same dense layer is first applied extract the most useful info from embedding layers
    model.add(TimeDistributed(Dense(shape['nr_hidden'], use_bias=False)))
    model.add(Bidirectional(LSTM(shape['nr_hidden'],
                                 recurrent_dropout=settings['dropout'],
                                 dropout=settings['dropout'])))
    model.add(Dense(shape['nr_class'], activation='sigmoid'))
    model.compile(optimizer=Adam(lr=settings['lr']), loss='categorical_crossentropy',
		  metrics=['accuracy'])
    return model

def get_features(docs, max_length):
    docs = list(docs)
    Xs = numpy.zeros((len(docs), max_length), dtype='int32')
    for i, doc in tqdm(enumerate(docs),total=len(docs)):
        j = 0
        for token in doc:
            ##rever to word vector
            vector_id = token.vocab.vectors.find(key=token.orth)
            if vector_id >= 0:
                Xs[i, j] = vector_id
            else:
                Xs[i, j] = 0
            j += 1
            if j >= max_length:
                break
    return Xs

# subset sra data 

In [5]:
srsWithText=srsS.index.get_level_values(0).unique()

In [6]:
technical_meta_data_df_sub=technical_meta_data_df[technical_meta_data_df.Sample.isin(srsWithText)]

### subset data based on entity types 

In [7]:
grouping_df=pd.read_csv('./Results/grouping.csv')

In [8]:
grouping_df['GroupName'].value_counts()

cell type          16
SCIENTIFIC_NAME    11
geo_loc_name       10
sex                10
disease            10
genotype            7
Name: GroupName, dtype: int64

### cap contribution by each study

In [9]:
max_sample_per_study_n=100

In [10]:
myAttribs=grouping_df.Attributes.unique()

In [11]:
subsampledSrs=technical_meta_data_df.groupby('Study').head(n=max_sample_per_study_n)['Sample']
print ('classify: {}'.format(", ".join(myAttribs)))
m=srsS.index.get_level_values(0).isin(subsampledSrs.values)
m1=srsS.index.get_level_values(1).isin(myAttribs)
srsS_subS=srsS[m&m1]

classify: SCIENTIFIC_NAME, organism, Organism, host scientific name, Species, host, specific host, host_scientific_name, host organism, nat-host, specific_host, sex, host_sex, Sex, gender, host sex, host_gender, Gender, host-sex, SEX, GENDER, genotype, genotype/variation, plant genotype, mutant, mutation, phenotype, host_genotype, disease, tumor type, diagnosis, disease state, DiseaseState, cancer type, tumor, clinical history, disease status, cell description, cell type, cell_type, source cell type, cell types, source_name, cell-type, CellType, cell subtype, biomaterial_type, progenitor cell type, tissue/cell type, DIFFERENTIATION_STAGE, cell, differentiation status, cell line source, geo_loc_name, geographic location, geo loc name, geographic location (country and/or sea, region), geographic location (country and/or sea,region), country, "geographic location (country and/or sea,region)", birth_location, geographic location (country and/or sea), Geo_loc_name


In [12]:
srsS_subS.head()

SRS1024493  SCIENTIFIC_NAME                  Camellia oleifera
            geo_loc_name       China: 856 m, Jinggang Mountain
            sex                                  hermaphrodite
SRS568274   SCIENTIFIC_NAME    Escherichia coli 2-316-03_S3_C1
            country                                   Tanzania
dtype: object

###  

In [13]:
filterTextL=['not collected','not applicable','missing','n[/]?a','unknown']

filterTextRegex="|".join(map(lambda myStr:'(?:{})'.format(myStr),filterTextL))

In [14]:
srs_filterM=srsS_subS.str.contains(filterTextRegex,case=False)
srsS_subS=srsS_subS[~srs_filterM]

In [15]:
myDf=srsS_subS.reset_index()
myDf.columns=['srs','attrib','sentence']

In [16]:
myDf['sentence']=myDf['sentence'].str.replace('_',' ')

# split data into training and  testing randomly by study levels

It is split by study level to avoid overgeneralization

In [17]:
train_test_ratio=0.8

In [18]:
study_S=technical_meta_data_df_sub['Study'].drop_duplicates()
print (len(study_S))
myNStudies=len(study_S)
train_n=int((myNStudies*train_test_ratio))
train_studies=study_S.sample(n=train_n,random_state=0).values
technical_meta_data_df_sub['Train']=technical_meta_data_df_sub['Study'].isin(train_studies).values

106722


/cellar/users/btsui/anaconda3/envs/deep_nlp_cpu/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


slice out the training and testing SRS ids

In [19]:
train_mask=technical_meta_data_df_sub['Train']
train_samples=technical_meta_data_df_sub['Sample'][train_mask].values
test_samples=technical_meta_data_df_sub['Sample'][~train_mask].values

### rename the attrib based on the grouped class name

In [20]:
myDf['orig_attrib']=myDf['attrib']
AttribToGroupNameS=grouping_df.groupby('Attributes')['GroupName'].first()
myDf['attrib']=AttribToGroupNameS[myDf['orig_attrib'].values].values

In [21]:
### set training and testing within the dataframe
all_train_df=myDf[myDf.srs.isin(train_samples)]
g=all_train_df.groupby('attrib')
print (g.size())

attrib
SCIENTIFIC_NAME    1008762
cell type           300054
disease              23064
genotype             87836
geo_loc_name        266511
sex                 149206
dtype: int64


### subsetting training and testing data 
shuffle the training data to make sure the model isn't learning the ordering of the data

In [22]:
cap_size=50000#20000#1000#50000
train_df=all_train_df.sample(n=all_train_df.shape[0]).groupby('attrib').head(n=cap_size)
all_test_df=myDf[myDf.srs.isin(test_samples)]
test_df=all_test_df.sample(n=all_test_df.shape[0]).groupby('attrib').head(cap_size)

### set up LSTM model

In [23]:
nr_hidden=64 
max_length=7#, #95% percentile of training phrase length from NCIT
dropout=0.5
learn_rate=0.001#, # General NN config
nb_epoch=1#
batch_size=100

use the sklearn encoder going back and forth between classes in string format and integer format

In [24]:
classNames=AttribToGroupNameS.unique()

In [25]:
le = preprocessing.LabelEncoder()
le.fit(classNames)
nr_classes=len(le.classes_)

In [26]:
lstm_shape={'nr_hidden': 64, 'max_length': max_length, 'nr_class': nr_classes}
lstm_settings={'dropout': 0.5, 'lr': 0.001}

In [27]:
embeddings = nlp.vocab.vectors.data

### transform list of freetexts into a matrix of word embeddings

In [28]:
train_df.attrib.value_counts()

cell type          50000
genotype           50000
SCIENTIFIC_NAME    50000
geo_loc_name       50000
sex                50000
disease            23064
Name: attrib, dtype: int64

In [29]:
train_texts=train_df.sentence.tolist()
dev_texts=test_df.sentence.tolist()

train_labels=keras.utils.to_categorical(
    le.transform(train_df.attrib.values))
dev_labels=keras.utils.to_categorical(le.transform(test_df.attrib.values))

train_docs = list(tqdm(nlp.pipe(train_texts,n_threads=32),total=len(train_texts)))
dev_docs = list(tqdm(nlp.pipe(dev_texts,n_threads=32),total=len(dev_texts)))

train_X = get_features(train_docs, lstm_shape['max_length'])
dev_X = get_features(dev_docs, lstm_shape['max_length'])

100%|██████████| 217788/217788 [00:01<00:00, 156111.33it/s]


#### Model training


#26 mins per epoch, on a machine with 48 threads, might want to use GPU machine bordeaux for this 


In [30]:
model_path='./model/lstm.h5'
log_dir="logs/"

In [31]:
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint
from IPython.utils import io

checkpoint = ModelCheckpoint(model_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tf_callBack=keras.callbacks.TensorBoard(log_dir=log_dir)
call_backs = [tf_callBack,checkpoint]

In [32]:
numpy.save('./model/classes.npy', le.classes_)

In [ ]:
model = compile_lstm(embeddings, lstm_shape, lstm_settings)

In [ ]:
#%%capture keras_stdout
lstm=model.fit(train_X, train_labels, validation_data=(dev_X, dev_labels),
          nb_epoch=nb_epoch,verbose=1, batch_size=batch_size,callbacks=call_backs)

In [ ]:
#0.93458
#print (keras_stdout)
#ValueError: Error when checking target: expected dense_4 to have shape (46,) but got array with shape (41,)


In [ ]:
#%%capture MyTest
#print ("asdasd")


### see how training accuracy improve as the amount of data increase

In [ ]:
keras_stdout_str=keras_stdout.stdout

In [ ]:
#pd.Series(keras_stdout_str.split('\r')).str.replace('\x08','').values

In [ ]:
trainingAcc=pd.Series(keras_stdout_str.split('\r')).str.replace('\x08','').str.extract('acc: (\d+.\d+)').dropna()[0].astype(float)

In [ ]:
#trainingAcc

In [ ]:
trainingAcc.index=trainingAcc.index*batch_size/1000

In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
np.arange(0,0.9,0.1)

In [ ]:
fig,ax=plt.subplots(figsize=(4,3))
trainingAcc.plot(ax=ax)
#ax.set_ylabel('#')
ax.set_xlabel('# of data point trained (thousands)')
ax.set_ylabel('Accuracy')
ax.set_yticks(np.arange(0.1,1.1,0.1))
ax.get_xaxis().set_major_formatter(
    matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
ax.figure.savefig('./Results/Figures/data_vol__accuracy.pdf')
ax.figure.savefig('./Results/Figures/data_vol__accuracy.png',dpi=300)

### evaluate model

In [ ]:
from keras.models import load_model


In [ ]:
#model=load_model(model_path)

In [ ]:
lstm=load_model('./model/lstm.h5')

In [ ]:
#loss_history = lstm.history['loss']
#numpy_loss_history = numpy.array(loss_history)
#numpy.savetxt("loss_history.txt", numpy_loss_history, delimiter=",")

In [ ]:
val_docs = list(tqdm(nlp.pipe(dev_texts,),total=len(dev_texts)))
val_X=get_features(val_docs,lstm_shape['max_length'])
%time predictM=lstm.model.predict_proba(val_X)

In [ ]:
probaDf_multI=pd.DataFrame(data=predictM,columns=le.classes_)

In [ ]:
probaDf_multI.index=pd.MultiIndex.from_arrays([test_df.attrib.values,dev_texts],names=['entity','text'])

In [ ]:
probaDf=probaDf_multI.reset_index()

In [ ]:
predicted_types=probaDf.loc[:,probaDf.columns.isin(myAttribs)].idxmax(axis=1)


In [ ]:
print (((probaDf.entity==predicted_types).mean()))

### show contingency tables

In [ ]:
#filterTextL=['not collected','not applicable','missing','n[/]?a']

#filterTextRegex="|".join(map(lambda myStr:'(?:{})'.format(myStr),filterTextL))

In [ ]:
#m_cleanText=~probaDf_multI.index.get_level_values('text').str.contains(filterTextRegex,case=False)

In [ ]:
inEvalDf=probaDf_multI#[m_cleanText]

In [ ]:
minProbThreshold=0.2
confidence_mask=inEvalDf.max(axis=1)>=minProbThreshold
tmpDf3=inEvalDf[confidence_mask].idxmax(axis=1).reset_index(name='predicted')

In [ ]:
tmpDf3.groupby('predicted').size()

In [ ]:
continDf=tmpDf3.groupby(['predicted','entity']).size().unstack().fillna(0)

In [ ]:
continDf.columns=continDf.columns.copy()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
attribVC_train=train_df.attrib.value_counts()#.to_frame()

In [ ]:
fig,ax=plt.subplots(figsize=(3,2))
#,cmap="Greens"
continDf=continDf.loc[attribVC_train.index,attribVC_train.index]
continDf.index=continDf.index.copy()
continDf.columns=continDf.columns.copy()
continDf.index.name='Predicted'
continDf.columns.name='Actual'
sns.heatmap(ax=ax,data=(continDf/continDf.sum(axis=0)).T*100,annot=True,cbar_kws={'label':'% of samples'},center=0.3,
           fmt='.1f')
fig.savefig('./Results/Figures/entity_contingency.pdf')
fig.savefig('./Results/Figures/entity_contingency.png',dpi=300)

In [ ]:
#sns.heatmap((continDf.T/continDf.sum(axis=1)).T,annot=True)

In [ ]:
#continDf#.sum(axis=0)


#### Performance of model in validation cohort

In [ ]:
fig,ax=plt.subplots(figsize=(4,3
                            ))
for myClass in le.classes_: 
    fpr,tpr,_=metrics.roc_curve((probaDf.entity==myClass),probaDf[myClass])    
    ax.plot(fpr,tpr,label="{myClass}  (AUC: {AUC})".format(myClass=myClass,AUC=str(metrics.auc(fpr,tpr))[:5]))
ax.set_xlabel('False positive rate')
ax.set_ylabel('True positive rate')
ax.legend()

In [ ]:
fig,ax=plt.subplots()
for myClass in le.classes_: 
    precision,recall,_=metrics.precision_recall_curve((probaDf.entity==myClass),probaDf[myClass])    
    ax.plot(recall, precision,label="{myClass}  (AUC: {AUC})".format(myClass=myClass,AUC=str(metrics.auc(recall,precision))[:5]))
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.legend()

In [ ]:
#probaDf

In [ ]:
#for myClass in le.classes_: 
#    print (myClass,metrics.f1_score((probaDf.entity==myClass),probaDf[myClass]))


### entities

In [ ]:
#inEvalDf

In [ ]:
entity_a=inEvalDf.index.get_level_values('entity')

In [ ]:
inTmpDf=inEvalDf.idxmax(axis=1).reset_index(name='predicted')

In [ ]:
inTmpDf['correct']=inTmpDf.entity==inTmpDf.predicted

In [ ]:
inTmpDf.to_csv('./model/pred.csv')

In [ ]:
inTmpDf[(inTmpDf.entity=='genotype')&(inTmpDf.predicted=='disease')]

In [ ]:
inTmpDf['correct'].mean()

In [ ]:
inTmpDf.groupby('entity')['correct'].mean().sort_values()

In [ ]:
#inEvalDf[entity_a=='breed'].sort_values('breed',ascending=False).index.get_level_values('text').value_counts()

In [ ]:
inEvalDf.head()

### calculated F1 score

In [ ]:
"""
``'binary'``:
        Only report results for the class specified by ``pos_label``.
        This is applicable only if targets (``y_{true,pred}``) are binary.
    ``'micro'``:
        Calculate metrics globally by counting the total true positives,
        false negatives and false positives.
    ``'macro'``:
        Calculate metrics for each label, and find their unweighted
        mean.  This does not take label imbalance into account.
    ``'weighted'``:
        Calculate metrics for each label, and find their average, weighted
        by support (the number of true instances for each label). This
        alters 'macro' to account for label imbalance; it can result in an
        F-score that is not between precision and recall.
    ``'samples'``:
        Calculate metrics for each instance, and find their average (only
        meaningful for multilabel classification where this differs from
        :func:`accuracy_score`).
"""
print( metrics.f1_score(y_true=inTmpDf['entity'],y_pred=inTmpDf['predicted'],average='weighted'))

# scratch

In [ ]:
asdadasd

### for a sentence split

In [ ]:
import nltk
from nltk.util import ngrams

sent="T cell is treated with LPS grow faster"
s=sent.split(' ')
n_gram=2
grams=list(map(lambda L:" ".join(L),list(ngrams(s,n_gram))))
print (grams)
val_docs = list(nlp.pipe(grams))
val_X=get_features(val_docs,lstm_shape['max_length'])
tmpDf=pd.DataFrame(data=lstm.model.predict_proba(val_X),columns=le.classes_,index=grams)
ax=sns.heatmap(tmpDf,cbar_kws={'label': 'Emitted probability'},annot=True)
ax.set_ylabel('')
ax.set_title('{} grams'.format(n_gram))

In [ ]:
val_X=get_features(val_docs,lstm_shape['max_length'])

In [ ]:
!echo $PWD/Data/DEEP_NLP/NLP_spacy/keras_on_sra_data_v2.ipynb

In [ ]:


tmpS3=srsS[srsS.index.get_level_values(1)=='cur_land_use']

In [ ]:
len(tmpS3.value_counts())

In [ ]:
#no data: biomass,chem_mutagen
srsS.head()

In [ ]:
def most_similar(word):
    by_similarity = sorted(word.vocab, key=lambda w: word.similarity(w), reverse=True)
    return [w.orth_ for w in by_similarity[:10]]

In [ ]:
#most_similar(nlp.vocab['missing'])

In [ ]:
train_df['sentence'][train_df['attrib']=='sex'].value_counts()

In [ ]:
'knockdown' in nlp.vocab

In [ ]:
train_df[train_df['attrib']=='cell type'].sentence.str.count(' ').quantile(0.95)

### check for threshold

In [ ]:
predDf=probaDf_multI.stack().reset_index()

In [ ]:
predDf.columns=['entity','text','predicted','proba']

In [ ]:
predDf['match']=predDf.entity==predDf.predicted

In [ ]:
sns.boxplot(data=predDf,x='entity',y='proba',hue='match',showfliers=False)

In [ ]:
predDf[predDf['match']].groupby(['entity'])['proba'].median().to_dict()

In [ ]:
predDf.groupby('proba')

In [ ]:
all_train_df_sub=all_train_df[all_train_df.attrib=='SCIENTIFIC_NAME']

In [ ]:
senVC=all_train_df_sub.sentence.value_counts()

In [ ]:
#senVC[senVC.index.str.contains('mouse')]

In [ ]:
takeOnlyTopXTest=1000
srsTmpDf=srsS_subS.reset_index()
srsTmpDf.columns=['SRS','Attrib','Text']
balancedDf=srsTmpDf.groupby(['Attrib','Text']).head(n=takeOnlyTopXTest)

srsS_subS=balancedDf.set_index(['SRS','Attrib'])['Text']

In [ ]:
from nltk.corpus import stopwords


In [ ]:
stopwords.words('english')